In [1]:
%pip install torch
%pip install transformers
%pip install sklearn
%pip install ipywidgets
%pip install IProgress

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [5]:

import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf
        tf.random.set_seed(seed)

set_seed(1)

dataset = load_dataset('csv', data_files={'train': ['data/train.csv'],'valid':['data/val.csv']})

# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

# def read_20newsgroups(test_size=0.2):
#     # download & load 20newsgroups dataset from sklearn's repos
#     dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
#     documents = dataset.data
#     labels = dataset.target
#     # split into training & testing a return data as well as label names
#     return train_test_split(documents, labels, test_size=test_size), dataset.target_names

# call the function
# (train_texts, valid_texts, train_labels, valid_labels), target_names = read_20newsgroups()
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(dataset['train']['ask'], truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(dataset['valid']['ask'], truncation=True, padding=True, max_length=max_length)

train_labels = dataset['train']['intent']
valid_labels = dataset['valid']['intent']


class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)
# load the model and pass to CUDA
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=68)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=False,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    save_total_limit=1,
    
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
# train the model


Using custom data configuration default-20922399493e373d
Reusing dataset csv (/home/juno/.cache/huggingface/datasets/csv/default-20922399493e373d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
trainer.train()
# evaluate the current model after training
trainer.evaluate()
# saving the fine tuned model & tokenizer
model_path = "20newsgroups_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

***** Running training *****
  Num examples = 5913
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1479


Step,Training Loss,Validation Loss,Accuracy
200,3.882500,3.301857,0.187817
400,3.071800,2.594037,0.443316
600,2.442700,2.041355,0.568528
800,2.085900,1.652498,0.631134
1000,1.632200,1.404051,0.688663
1200,1.375200,1.235210,0.736041
1400,1.228800,1.158741,0.751269


***** Running Evaluation *****
  Num examples = 591
  Batch size = 8
***** Running Evaluation *****
  Num examples = 591
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 591
  Batch size = 8
***** Running Evaluation *****
  Num examples = 591
  Batch size = 8
***** Running Evaluation *****
  Num examples = 591
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 591
  Batch size = 8
***** Running Evaluation *****
  Num examples = 591
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/mode

Configuration saved in 20newsgroups_model/config.json
Model weights saved in 20newsgroups_model/pytorch_model.bin
tokenizer config file saved in 20newsgroups_model/tokenizer_config.json
Special tokens file saved in 20newsgroups_model/special_tokens_map.json


('20newsgroups_model/tokenizer_config.json',
 '20newsgroups_model/special_tokens_map.json',
 '20newsgroups_model/vocab.txt',
 '20newsgroups_model/added_tokens.json',
 '20newsgroups_model/tokenizer.json')